In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l

# 词的相似性和类比任务

# 1. 加载预训练词向量

In [2]:
# 配置d2l的GloVe和wiki英文数据集地址和校验码
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

In [3]:
class TokenEmbedding:
    """GloVe嵌入类：负责加载和查询词向量"""
    def __init__(self, embedding_name):
        # 加载embedding，获取token和对应向量
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0 # 未知词的索引
        # 建立token到索引的映射
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    # 读取词表和向量
    def _load_embedding(self, embedding_name):
        # 实际加载embedding的函数
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe网站：https://nlp.stanford.edu/projects/glove/
        # fastText网站：https://fasttext.cc/
        # 读取vec.txt文件（GloVe的格式）
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # 跳过如fastText的首行（标题）等特殊行，只加入正常向量
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        # unk向量为0
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        # 支持通过token列表直接取向量
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [4]:
# 加载glove.6b.50d词向量
glove_6b50d = TokenEmbedding('glove.6b.50d')

In [5]:
# 词表大小
len(glove_6b50d)

400001

In [6]:
# 查词下标和词
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

# 2. 应用预训练词向量

## 2.1. 词相似度

In [7]:
# 计算向量x与矩阵W所有行的余弦相似度，返回前k个最大值下标和分数
def knn(W, x, k):
    # 增加1e-9以获得数值稳定性
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

In [8]:
def get_similar_tokens(query_token, k, embed):
    # 输出与query_token最相似的前k个词（除自身外)
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # 排除输入词，跳过自身
        print(f'{embed.idx_to_token[int(i)]}：cosine相似度={float(c):.3f}')

In [9]:
# 示例：找chip最相似的3个词
get_similar_tokens('chip', 3, glove_6b50d)

chips：cosine相似度=0.856
intel：cosine相似度=0.749
electronics：cosine相似度=0.749


In [10]:
get_similar_tokens('baby', 3, glove_6b50d)

babies：cosine相似度=0.839
boy：cosine相似度=0.800
girl：cosine相似度=0.792


In [11]:
get_similar_tokens('beautiful', 3, glove_6b50d)

lovely：cosine相似度=0.921
gorgeous：cosine相似度=0.893
wonderful：cosine相似度=0.830


## 2.2. 词类比

In [12]:
# 类比推理：token_a : token_b = token_c : ?
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    # b - a + c
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    # 返回最相近的词
    return embed.idx_to_token[int(topk[0])]

In [13]:
# 词类比：man : woman = son : ?
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [14]:
# 类比：中国-北京-日本-？
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

In [15]:
# 类比：bad - worst + big = ?
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

In [16]:
# 类比：do - did + go = ?
get_analogy('do', 'did', 'go', glove_6b50d)

'went'